In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/311_Service_Requests_from_2010_to_Present.csv")

/var/folders/4b/z0w6x5_n59g_x_9s3qnbx_p80000gn/T/ipykernel_57606/1050216877.py:4: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/311_Service_Requests_from_2010_to_Present.csv")


In [2]:
df.drop(['Unique Key', 'Location', 'Agency Name'], axis=1, inplace=True)
df.drop(['Bridge Highway Segment', 'Road Ramp', 'Bridge Highway Direction', 'Bridge Highway Name', 'Taxi Pick Up Location', 'Taxi Company Borough', 'Vehicle Type', 'Due Date', 'Facility Type'], axis=1, inplace=True)
df.drop(['Zip Codes', 'Community Districts', 'Borough Boundaries', 'City Council Districts'], axis=1, inplace=True)
df.drop(['Street Name', 'BBL', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Status', 'Community Board', 'Cross Street 1', 'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2', 'Landmark', 'Park Borough'], axis=1, inplace=True)

In [3]:
df.dtypes

Created Date                       object
Closed Date                        object
Agency                             object
Complaint Type                     object
Descriptor                         object
Location Type                      object
Incident Zip                      float64
Incident Address                   object
Address Type                       object
City                               object
Resolution Description             object
Resolution Action Updated Date     object
Borough                            object
Open Data Channel Type             object
Park Facility Name                 object
Latitude                          float64
Longitude                         float64
Police Precincts                  float64
dtype: object

In [4]:
df.describe()

,Incident Zip,Latitude,Longitude,Police Precincts
count,232946.000000,231674.000000,231674.000000,231638.000000
mean,10811.249805,40.735881,-73.924741,41.243164
std,627.407671,0.087520,0.076771,20.439052
min,7071.000000,40.498949,-74.252766,1.000000
25%,10451.000000,40.669770,-73.967920,26.000000
50%,11201.000000,40.730215,-73.927349,40.000000
75%,11233.000000,40.817920,-73.879641,60.000000
max,94402.000000,40.912468,-73.700736,77.000000


In [5]:
count_low = df[df['Incident Zip'] <= 10000.000000]['Incident Zip'].count()
count_high = df[df['Incident Zip'] >= 11697]['Incident Zip'].count()
print(count_low)
print(count_high)

76
19


In [6]:
df.drop(df[df['Incident Zip'] <= 10000.000000].index, inplace=True)
df.drop(df[df['Incident Zip'] >= 11697].index, inplace=True)

In [7]:
# fill null values with 'Other'
df['Location Type'] = df['Location Type'].fillna('Other')

# change all 'Other (Explain Below)' to 'Other' because they are the same thing
df['Location Type'] = df['Location Type'].replace('Other (Explain Below)', 'Other')

# fill null values with 'Other'
df['Descriptor'] = df['Descriptor'].fillna('Other')

# change all versions of other to uniform 'Other'
df['Descriptor'] = df['Descriptor'].replace({'Other (Explain Below)': 'Other',
                                             'Other (complaint details)': 'Other',
                                             'Other/Unknown': 'Other'})

df["Address Type"].value_counts(dropna=False)

# fill null values with 'UNRECOGNIZED'
df['Address Type'] = df['Address Type'].fillna('UNRECOGNIZED')

df['Park Facility Name'] = df['Park Facility Name'].fillna('Unspecified')

df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [8]:
df['Resolution Action Updated Date'] = pd.to_datetime(df['Resolution Action Updated Date'])
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'], errors='coerce')

In [9]:
# Check if there are any instances where the "Closed Date" is earlier than the "Created Date"
closed_earlier = df[df['Closed Date'] < df['Created Date']]

if closed_earlier.empty:
    print("No instances where 'Closed Date' is earlier than 'Created Date'")
else:
    print("There are instances where 'Closed Date' is earlier than 'Created Date'")

There are instances where 'Closed Date' is earlier than 'Created Date'


In [10]:
# creates an index containing all of examples where closed date is before created data and drops those columns
index_to_drop = df.index[df['Closed Date'] < df['Created Date']].tolist()
df.drop(index_to_drop, inplace=True)

In [11]:
print((df.isnull().sum() * 100)/ len(df))

Created Date                      0.000000
Closed Date                       6.293567
Agency                            0.000000
Complaint Type                    0.000000
Descriptor                        0.000000
Location Type                     0.000000
Incident Zip                      1.268319
Incident Address                  4.771924
Address Type                      0.000000
City                              4.338383
Resolution Description            1.264494
Resolution Action Updated Date    1.136132
Borough                           0.000000
Open Data Channel Type            0.000000
Park Facility Name                0.000000
Latitude                          1.802169
Longitude                         1.802169
Police Precincts                  1.817471
dtype: float64


In [12]:
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

In [13]:
df['duration'] = df['Closed Date'] - df['Created Date']

# convert the resulting timedelta object to a numerical value in seconds
df['duration_seconds'] = df['duration'].dt.total_seconds()

df['duration_hours'] = df['duration'].dt.total_seconds() / 3600

df['duration_days'] = df['duration'].dt.days

# Create two new columns - one for the day of the week and one for the type of day (weekday/weekend):
df['day_of_week'] = df['Created Date'].dt.dayofweek
df['day_type'] = np.where(df['day_of_week'] < 5, 'weekday', 'weekend')

In [14]:
df['duration_hours'] = df['duration_hours'].fillna(168)

In [15]:
from uszipcode import SearchEngine
import pandas as pd

sr = SearchEngine()

borough_zipcodes = {
    'Bronx': [str(zipcode) for zipcode in range(10451, 10476)],
    'Brooklyn': [str(zipcode) for zipcode in range(11201, 11257)],
    'Manhattan': [str(zipcode) for zipcode in range(10001, 10283)],
    'Queens': [str(zipcode) for zipcode in range(11004, 11110)] + [str(zipcode) for zipcode in range(11351, 11698)],
    'Staten Island': [str(zipcode) for zipcode in range(10301, 10315)]
}

zip_info_list = []

for borough, zipcodes in borough_zipcodes.items():
    for zipcode in zipcodes:
        zip_info = sr.by_zipcode(zipcode)
        if zip_info is not None:
            zip_info_dict = zip_info.__dict__
            zip_info_dict['borough'] = borough
            zip_info_list.append(zip_info_dict)

df1 = pd.DataFrame(zip_info_list)

df1.to_csv('zip_codes.csv')

In [16]:
df_zip = pd.read_csv("/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/zip_codes.csv")

In [17]:
df_zip.head()

,Unnamed: 0,_sa_instance_state,lat,water_area_in_sqmi,bounds_east,zipcode_type,lng,housing_units,bounds_north,zipcode,...,median_home_value,common_city_list,population,median_household_income,county,population_density,bounds_west,state,land_area_in_sqmi,borough
0,0,<sqlalchemy.orm.state.InstanceState object at ...,40.82,0.0,-73.908709,STANDARD,-73.93,17779.0,40.830915,10451,...,170600.0,['Bronx'],45713.0,26754.0,Bronx County,45043.0,-73.932941,NY,1.01,Bronx
1,1,<sqlalchemy.orm.state.InstanceState object at ...,40.84,0.0,-73.911049,STANDARD,-73.92,25866.0,40.846520,10452,...,402800.0,['Bronx'],75371.0,25979.0,Bronx County,76548.0,-73.933406,NY,0.98,Bronx
2,2,<sqlalchemy.orm.state.InstanceState object at ...,40.85,0.0,-73.899949,STANDARD,-73.91,27018.0,40.860601,10453,...,445200.0,['Bronx'],78309.0,25470.0,Bronx County,85491.0,-73.928242,NY,0.92,Bronx
3,3,<sqlalchemy.orm.state.InstanceState object at ...,40.81,0.0,-73.901892,STANDARD,-73.92,12772.0,40.813731,10454,...,386000.0,['Bronx'],37337.0,20232.0,Bronx County,35820.0,-73.931681,NY,1.04,Bronx
4,4,<sqlalchemy.orm.state.InstanceState object at ...,40.81,0.0,-73.895158,STANDARD,-73.91,13851.0,40.821277,10455,...,318600.0,['Bronx'],39665.0,22609.0,Bronx County,55696.0,-73.922088,NY,0.71,Bronx


In [18]:
df_zip.drop(['Unnamed: 0', '_sa_instance_state', 'lat', 'water_area_in_sqmi', 'bounds_east', 'zipcode_type', 'lng', 'bounds_north', 'common_city_list', 'county', 'bounds_west', 'state', 'land_area_in_sqmi', 'borough', 'timezone', 'bounds_south', 'major_city', 'radius_in_miles', 'post_office_city', 'area_code_list'], axis=1, inplace=True)

In [19]:
df_zip.head()

,housing_units,zipcode,occupied_housing_units,median_home_value,population,median_household_income,population_density
0,17779.0,10451,17106.0,170600.0,45713.0,26754.0,45043.0
1,25866.0,10452,24796.0,402800.0,75371.0,25979.0,76548.0
2,27018.0,10453,25307.0,445200.0,78309.0,25470.0,85491.0
3,12772.0,10454,11978.0,386000.0,37337.0,20232.0,35820.0
4,13851.0,10455,12880.0,318600.0,39665.0,22609.0,55696.0


In [20]:
# merge df and df_zip on 'Incident Zip' in df and 'zipcode' in df_zip
df = pd.merge(df, df_zip, left_on='Incident Zip', right_on='zipcode', how='left')

# drop the 'zipcode' column from the merged dataframe
df = df.drop('zipcode', axis=1)

In [21]:
df = df[df['Borough'] != 'unspecified']

In [22]:
print((df.isnull().sum() * 100)/ len(df))

Created Date                      0.000000
Closed Date                       6.341597
Agency                            0.000000
Complaint Type                    0.000000
Descriptor                        0.000000
Location Type                     0.000000
Incident Zip                      0.480279
Incident Address                  4.150863
Address Type                      0.000000
City                              3.286100
Resolution Description            1.253582
Resolution Action Updated Date    1.141747
Borough                           0.000000
Open Data Channel Type            0.000000
Park Facility Name                0.000000
Latitude                          0.000000
Longitude                         0.000000
Police Precincts                  0.015171
duration                          6.341597
duration_seconds                  6.341597
duration_hours                    0.000000
duration_days                     6.341597
day_of_week                       0.000000
day_type   

In [23]:
# Replace all strings containing 'noise' with 'Noise'
df['Complaint Type'] = df['Complaint Type'].str.replace(r'.*noise.*', 'noise', regex=True)

In [24]:
# descriptor_counts = df['Complaint Type'].value_counts()
# num_single_counts = (descriptor_counts == 1).sum()
# print(f'There are {num_single_counts} values in the Descriptor column with a value count of 1')

In [25]:
# descriptor_value_counts = df['Descriptor'].value_counts()

# descriptor_stats = descriptor_value_counts.describe()

# print(descriptor_stats)

In [26]:
counts = df['Descriptor'].value_counts()
to_remove = counts[counts < 10].index
df = df[~df['Descriptor'].isin(to_remove)]

counts2 = df['Complaint Type'].value_counts()
to_remove2 = counts2[counts2 < 10].index
df = df[~df['Complaint Type'].isin(to_remove2)]

In [27]:
df['Borough'].value_counts()

brooklyn         71363
queens           52037
bronx            49869
manhattan        46485
staten island    10012
Name: Borough, dtype: int64

In [28]:
df.to_csv('/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/cleaned.csv', index=False)

<!-- ---
title: "Investigating 311 Complaints in NYC"
subtitle: "A Data Science Research Proposal"
author:
- name: Your Name
  email: your.email@example.com
  twitter: your_twitter_handle
output:
  quarto::quarto_presentation:
    theme: sky
    highlight: tango
---

## Introduction

This research proposal aims to investigate the relationship between specific types of 311 complaints and the boroughs they are most common in, as well as the demographic characteristics of those boroughs. This research is important as it can help identify potential areas for improvement in service delivery and policy development in New York City. Previous studies have analyzed the relationship between 311 complaints and various demographic factors such as income, race, and education level (e.g. “Structure of 311 service requests as a signature of urban location”). However, there is a need to further explore the variations in 311 complaints across different boroughs.

## Cleaning Data

The following code was used to clean the data:

```{code-block, engine='python'} -->